In [6]:
cd /Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a

/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a


In [7]:
# change to correct directory

In [8]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt
import GPy

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker
import random

import pyDOE
import random
from scipy.stats import norm

from sklearn.ensemble import RandomForestRegressor

In [9]:
# chanload the COE dataset

dataset = pd.read_excel(r'/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a/C2a learning study.xlsx')
X = dataset.iloc[:,0:5]

X_run = X.groupby(['CsPbI', 'MAPbI', 'FAPbI'])['Instability index'].agg(lambda x: x.unique().mean())
X_run = (X_run.to_frame()).reset_index()
X_run
# verify that the data is succesfully loaded


,CsPbI,MAPbI,FAPbI,Instability index
0,0.00,0.00,1.00,4.929210e+05
1,0.00,0.25,0.75,9.480030e+05
2,0.00,0.50,0.50,1.129496e+06
3,0.00,0.75,0.25,1.290847e+06
4,0.00,1.00,0.00,1.452513e+06
...,...,...,...,...
89,0.88,0.12,0.00,4.048450e+05
90,0.89,0.11,0.00,4.819935e+05
91,0.91,0.08,0.01,4.822180e+05
92,0.96,0.00,0.04,3.497070e+05


In [10]:
materials = ['CsPbI', 'MAPbI', 'FAPbI']
X_feature = X_run[['CsPbI', 'MAPbI', 'FAPbI']].values

X_Cs = np.array(X_run['CsPbI'].values)
X_FA = np.array(X_run['FAPbI'].values)
X_MA = np.array(X_run['MAPbI'].values)
X_instability = np.array(X_run['Instability index'].values)

# bounds = [None for j in range(len(materials))]
# for j in range(len(materials)):
#     bounds[j] = {'name': materials[j], 'type': 'continuous', 'domain': (0,1)}

# tolerance_factor_constraint = '((167*x[:,0]+217*x[:,1]+253*x[:,2])+220)/(1.4142*(119+220))'
# constraints_baseline = [{'name': 'constr_A_max', 'constraint': 'x[:,0] +x[:,1] +x[:,2]- ' + str(1)},
#                         {'name': 'constr_A_min', 'constraint': str(0.99)+'-x[:,0] -x[:,1]-x[:,2]'},
#                         {'name': 'G_tolerance_min', 'constraint': '0.80 - ' + tolerance_factor_constraint},
#                         {'name': 'G_tolerance_max', 'constraint': tolerance_factor_constraint + ' - 1'}
#                        ]


In [5]:
def cross_valid(X_feature, y_, kernel):
    assert len(X_feature) == len(y_)
    n = len(X_feature)
    
    seed_list = [74,660,861,1526,2,151,19,7,33,4021]
    total_mae = []    
    n_hold = int(np.round(n * 0.10))
#     n_hold = 1
    
    for s in seed_list:
        print('initializing seed = ' +str(seed_list.index(s)))
        random.seed(s)
        
        indices = list(np.arange(n))
        
        index_ = random.sample(indices, n_hold)
        
        X_test = []
        y_test = []
        
        for i in index_:
            X_test.append(X_feature[i])
            y_test.append(y_[i])
            indices.remove(i)
        
        X_train = X_feature[indices]
        y_train = y_[indices]
        
        y_train_normalized, raw_mean, raw_std = normalize_Y(y_train)
        
        GP_model = GPy.models.GPRegression(X = np.array(X_train), 
                                           Y = np.array([[i] for i in y_train_normalized]), 
                                           kernel= kernel,
                                           noise_var = 0.01
                                          )

        GP_model.optimize_restarts(num_restarts=10,
                                   parallel = True,
                                   robust = True,
                                   optimizer = 'bfgs',
                                   max_iters=100,
                                   verbose = False)
        
        
        
        normalized_pred_mean = GP_model.predict(np.array(X_test))[0][:,0]
        raw_pred_mean = normalized_pred_mean * raw_std + raw_mean
        mae_ = np.sqrt((y_test - raw_pred_mean)**2)
        total_mae.append(np.sum(mae_))
        
    return np.mean(np.array(total_mae))



In [48]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [109]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(mape)
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X_feature, X_instability, test_size=0.15, random_state=42)

In [140]:
param_grid = {'n_estimators': [5, 50],
              'max_depth': [None]}

In [141]:
rf = RandomForestRegressor(random_state = 42)
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.4s finished
 /opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [None], 'n_estimators'

In [142]:
grid_search.best_params_

{'max_depth': None, 'n_estimators': 50}

In [143]:
evaluate(grid_search.best_estimator_, X_test, y_test)

Model Performance
Average Error: 109772.5200 degrees.
Accuracy = 66.56%.


66.55582054428785

In [144]:
grid_search.best_estimator_.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [145]:
rf_base = RandomForestRegressor(n_estimators=50, random_state = 42)

In [146]:
rf_base.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [147]:
rf_base.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [148]:
evaluate(rf_base, X_test, y_test)

Model Performance
Average Error: 109772.5200 degrees.
Accuracy = 66.56%.


66.55582054428785

In [ ]:
random_grid = {'bootstrap': [False],
               'max_depth': [2, 4, 6, 8, 10, 20, None],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 4, 8],
               'n_estimators': [5, 10, 20, 30, 40, 50,100]
              }

rf = RandomForestRegressor(random_state = 42)
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 500, 
                               cv = 5, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)

rf_random.fit(X_train, y_train)

rf_random.best_params_

evaluate(rf_random.best_estimator_, X_test, y_test)